In [1]:
import pandas as pd

url = 'https://github.com/mattharrison/datasets/raw/master/data/''vehicles.csv.zip'
df = pd.read_csv(url, dtype_backend='pyarrow', engine='pyarrow')
pd.set_option('display.max_columns', None)

def gt20(val):
    return val > 20

city_mpg = df.city08
highway_mpg = df.highway08
make = df.make

In [2]:
make

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [3]:
make.astype(str)

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: object

In [4]:
import pyarrow as pa
string_pa = pd.ArrowDtype(pa.string())
make.astype(str).astype(string_pa)

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [5]:
'Ford'.lower()
'Hello'.lower()

'hello'

In [6]:
make.str.lower()

0        alfa romeo
1           ferrari
2             dodge
3             dodge
4            subaru
            ...    
41139        subaru
41140        subaru
41141        subaru
41142        subaru
41143        subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [7]:
'Alpha Romeo'.find('A')

0

In [8]:
make.str.find('A')

0         0
1        -1
2        -1
3        -1
4        -1
         ..
41139    -1
41140    -1
41141    -1
41142    -1
41143    -1
Name: make, Length: 41144, dtype: int32[pyarrow]

In [9]:
# print(make.str.extract(r'([^a-z A-Z])')) ERROR

In [10]:
print(make.str.extract(r'(?P<non_alpha>[^a-z A-Z])'))

      non_alpha
0          <NA>
1          <NA>
2          <NA>
3          <NA>
4          <NA>
...         ...
41139      <NA>
41140      <NA>
41141      <NA>
41142      <NA>
41143      <NA>

[41144 rows x 1 columns]


In [11]:
(make
 .str.extract(r'(?P<non_alpha>[^a-z A-Z])',
              expand=False)
 .value_counts()
)

non_alpha
-    1727
.      46
,       9
Name: count, dtype: int64[pyarrow]

In [12]:
# If a column in a CSV file contains non-numeric characters, use the following code to find them:

col = make

(col
 .str.extract(r'(?P<non_num>[^0-9.])', expand=False)
 .value_counts()
)

non_num
C    5336
M    4833
F    3686
B    2796
G    2691
D    2679
P    2589
S    2234
T    2159
V    2001
H    1803
A    1610
N    1471
J    1435
L    1241
I     860
K     618
O     462
R     392
E     167
s      38
W      32
Y       8
Q       3
Name: count, dtype: int64[pyarrow]

In [13]:
import pyarrow as pa
string_pa = pd.ArrowDtype(pa.string())
age = pd.Series(['0-10', '11-15', '11-15', '61-65', '46-50'],
                 dtype=string_pa)
age

0     0-10
1    11-15
2    11-15
3    61-65
4    46-50
dtype: string[pyarrow]

In [14]:
age.str.split('-')

0     ['0' '10']
1    ['11' '15']
2    ['11' '15']
3    ['61' '65']
4    ['46' '50']
dtype: list<item: string>[pyarrow]

In [15]:
(age
 .str.split('-', expand=True)
 .iloc[:,0]
 .astype('int8[pyarrow]')
)

0     0
1    11
2    11
3    61
4    46
Name: 0, dtype: int8[pyarrow]

In [16]:
(age
 .str.slice(-2)
 .astype('int8[pyarrow]')
)

0    10
1    15
2    15
3    65
4    50
dtype: int8[pyarrow]

In [17]:
(age
 .str[-2:]
 .astype('int8[pyarrow]')
)

0    10
1    15
2    15
3    65
4    50
dtype: int8[pyarrow]

In [18]:
(age
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .mean(axis='columns')
)

0     5.0
1    13.0
2    13.0
3    63.0
4    48.0
dtype: double[pyarrow]

In [19]:
import random
def between(row):
    return random.randint(*row.values)

In [20]:
(age
 .str.split('-', expand=True)
 .astype(int)
 .apply(between, axis='columns')
)

0     7
1    13
2    13
3    64
4    48
dtype: int64

In [21]:
import numpy as np
print(age
      .str.split('-', expand=True)
      .rename(columns={0:'lower',1:'upper'})
      .astype('int8[pyarrow]')
      .assign(rand=np.random.rand(len(age)),
              age=lambda df_: (df_.lower + (df_.rand *
                                            (df_.upper - df_.lower)))
              .astype('int8[pyarrow]', errors='ignore')
             )
     )


   lower  upper      rand        age
0      0     10  0.396236   3.962362
1     11     15  0.893016  14.572063
2     11     15  0.390988  12.563951
3     61     65  0.205707  61.822827
4     46     50  0.503962  48.015849


In [22]:
import numpy as np # imports numpy
(age # uses the age bins shown here: age = pd.Series(['0-10', '11-15', '11-15', '61-65', '46-50'],
 .str.split('-', expand=True) # splits the bins by '-' into two columns
 .astype(int) # turns the output of the two columns into integers
 .pipe(lambda df_: pd.Series(np.random.randint(df_.iloc[:,0], # not yet sure what pipe does. # takes a random intiger and does something with iloc idk what the comma in iloc[:,0] does
                                               df_.iloc[:,1]), index = df_.index) # idk what [:,1] the comma does here, and does something with an index
      )
)

0     8
1    11
2    11
3    63
4    49
dtype: int64

In [23]:
age_100k = (age # takes age series
            .sample(100_000, replace=True, random_state=42) # creates a randomly sampled figures from age, 100,000 of them
            .reset_index(drop=True) # creates a natural index, as the original index values are kept from the source Series
           )
age_100k

0        61-65
1        46-50
2        11-15
3        46-50
4        46-50
         ...  
99995    11-15
99996     0-10
99997    46-50
99998    61-65
99999     0-10
Length: 100000, dtype: string[pyarrow]

In [24]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .pipe(lambda df_: pd.Series(np.random.randint(df_.iloc[:,0], 
                                               df_.iloc[:,1]), index =df_.index)
                            )
      )

12.4 ms ± 475 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .apply(between, axis='columns')
)

2.25 s ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .rename(columns={0:'lower', 1:'upper'})
 .astype('int8[pyarrow]')
 .assign(rand=np.random.rand(len(age_100k)),
         age=lambda df_: (df_.lower + (df_.rand * 
                                       (df_.upper - df_.lower)))
         .astype('int8[pyarrow]', errors='ignore')
        )
)

12.6 ms ± 374 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
(age_100k
 .str.split('-', expand=True) # splits into columns
 .rename(columns={0:'lower', 1:'upper'}) # renames the first column as lower, and second as upper
 .astype('int8[pyarrow]') # set type of intiger
 .assign(rand=np.random.rand(len(age_100k)), # creates rand column as decimals for 100 rows contained in age_100k 
         age=lambda df_: (df_.lower + (df_.rand *  ( # the lower age is added to the multiplication of the range differences with the random intiger found in in rand colum
                                       (df_.upper - df_.lower))) # finds the difference between lower and upper
         .astype('int8[pyarrow]', errors='ignore') # converts to int8 using PyArrow and ignores any values that can't be computed
        )
)
)

,lower,upper,rand,age
0,61,65,0.279029,62.116114
1,46,50,0.320900,47.283598
2,11,15,0.272636,12.090542
3,46,50,0.236211,46.944843
4,46,50,0.213092,46.852369
...,...,...,...,...
99995,11,15,0.735504,13.942014
99996,0,10,0.945133,9.451328
99997,46,50,0.730028,48.92011
99998,61,65,0.372781,62.491126


In [28]:
%load_ext Cython

In [29]:
%%cython
import random
def between_cy(row):
    return random.randint(*row.values)

In [30]:
%%cython
import random
import numpy as np
def between_cy3(x: np.int64, y: np.int64) -> np.int64:
    return random.randint(x,y)

In [31]:
(age # selects age series to run
 .str.split('-', expand=True) # Splits the ages into two collumns
 .astype(int) # converts all to intigers
 .apply(lambda row: between_cy3(row[0], row[1]), axis=1) # apply the between_cy3 function for first and second row, horizontally
)

0     4
1    14
2    15
3    65
4    50
dtype: int64

In [32]:
%%timeit
(age_100k # selects age series to run
 .str.split('-', expand=True) # Splits the ages into two collumns
 .astype(int) # converts all to intigers
 .apply(lambda row: between_cy3(row[0], row[1]), axis=1) # apply the between_cy3 function for first and second row, horizontally
)

400 ms ± 8.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%prun  -l 10 (age_100k.str.split('-', expand=True).astype(int) \
       .apply(lambda row: between_cy3(row[0], row[1]), axis=1))

         4658611 function calls (4658564 primitive calls) in 1.203 seconds

   Ordered by: internal time
   List reduced from 399 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   200000    0.172    0.000    0.603    0.000 series.py:1095(__getitem__)
1100351/1100345    0.106    0.000    0.159    0.000 {built-in method builtins.isinstance}
   100001    0.099    0.000    0.199    0.000 apply.py:1247(series_generator)
   200000    0.097    0.000    0.243    0.000 series.py:1220(_get_value)
   200000    0.073    0.000    0.087    0.000 base.py:3777(get_loc)
   200000    0.066    0.000    0.128    0.000 indexing.py:2765(check_dict_or_set_indexers)
        1    0.055    0.055    1.153    1.153 apply.py:1070(apply_series_generator)
   100000    0.053    0.000    0.139    0.000 random.py:291(randrange)
   100000    0.051    0.000    0.821    0.000 <string>:1(<lambda>)
   100000    0.048    0.000    0.056    0.000 managers.py:2065(set_valu

In [34]:
%%cython

import numpy as np
import random
def apply_between_cy4(x: np.ndarray[int],
                      y: np.ndarray[int]) -> np.ndarray[int]:
    res: np.ndarray[int] = np.empty(len(x), dtype='int32')
    i : int
    for i in range(len(x)):
        res[i] = random.randint(x[i], y[i])
    return res

In [35]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype('int8[pyarrow]')
 .pipe(lambda df_: apply_between_cy4(
     df_.iloc[:,0].to_numpy(dtype='int32'),
     df_.iloc[:,1].to_numpy(dtype='int32')))
)

139 ms ± 5.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%%cython
import numpy as np
import random
def apply_between_cy5(x: np.ndarray[int],
                      y: np.ndarray[int]) -> np.ndarray[long]:
    return np.random.randint(x,y)

In [37]:
%%timeit
(age_100k
 .str.split('-', expand=True)
 .astype(int)
 .pipe(lambda df_: apply_between_cy5(
     df_.iloc[:,0].to_numpy(dtype='int32'),
     df_.iloc[:,1].to_numpy(dtype='int32')))
 )

24.9 ms ± 393 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
# Optimising .apply with Numba

In [39]:
import numba
numba.__version__

'0.59.1'

In [40]:
import numba as nb
import numpy

@nb.jit(nb.int32[:](nb.int32[:], nb.int32[:]))
def between_nb(arr1, arr2):
    return numpy.random.randint(arr1, arr2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in method randint of numpy.random.mtrand.RandomState object at 0x10a820b40>) found for signature:
 
 >>> randint(array(int32, 1d, A), array(int32, 1d, A))
 
There are 6 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'np_randint_impl_1': File: numba/cpython/randomimpl.py: Line 568.
    With argument(s): '(array(int32, 1d, A), array(int32, 1d, A))':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: [1mtoo many positional arguments[0m[0m
  raised from /opt/anaconda3/lib/python3.12/site-packages/numba/core/typing/templates.py:784
[1m  - Of which 2 did not match due to:
  Overload of function 'randint': File: numba/cpython/randomimpl.py: Line 574.
    With argument(s): '(array(int32, 1d, A), array(int32, 1d, A))':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  

In [41]:
(age_100k
 .str.split('-', expand=True)
 .astype(int)
 .pipe(lambda df_: between_nb(df_.iloc[:,0].to_numpy(dtype='int32'),
                              df_.iloc[:,1].to_numpy(dtype='int32')))
)

NameError: name 'between_nb' is not defined

In [42]:
make.str.replace('A','Å')

0        Ålfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [43]:
make[make == 'A']

Series([], Name: make, dtype: string[pyarrow])

In [44]:
make.replace('A','Å')

0        Alfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [49]:
make.replace({'Audi': 'Åudi', 'Acura': 'Åcura',
             'Ashton Martin': 'Åshton Martin', 
             'Alfa Romeo': 'Ålfa Romeo'})

0        Ålfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [50]:
make.replace('A','Å', regex=True)

0        Ålfa Romeo
1           Ferrari
2             Dodge
3             Dodge
4            Subaru
            ...    
41139        Subaru
41140        Subaru
41141        Subaru
41142        Subaru
41143        Subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [51]:
# Exercises

In [52]:
# Using a string column, lowercase the values

make.str.lower()

0        alfa romeo
1           ferrari
2             dodge
3             dodge
4            subaru
            ...    
41139        subaru
41140        subaru
41141        subaru
41142        subaru
41143        subaru
Name: make, Length: 41144, dtype: string[pyarrow]

In [71]:
# Using a string column, slice out the first character.
 
# make[0] returns only the first item

make.str[0] # you need to go into the str function to select the application you want to do, and then [0] for selecting the characters

0        A
1        F
2        D
3        D
4        S
        ..
41139    S
41140    S
41141    S
41142    S
41143    S
Name: make, Length: 41144, dtype: string[pyarrow]

In [75]:
# Using a string column, slice out the last three characters.

make.str[-3:]

0        meo
1        ari
2        dge
3        dge
4        aru
        ... 
41139    aru
41140    aru
41141    aru
41142    aru
41143    aru
Name: make, Length: 41144, dtype: string[pyarrow]

In [106]:
# Create a series extracting the numeric values using a string column.

product_info = pd.Series([
    "Item#1001 - Price: $25.50",
    "SKU: A325, Qty: 10",
    "Ref: X9988, Units: 300",
    "Discounted: 15%",
    "Box ID 3030 - Weight: 5kg",
    "Product Z - ID9007 - 40pcs",
    "OldStock - 2017 batch",
    "Volume: 2.5L - Item 8891",
    "Wattage 60W - Model MZ22",
    "Serial 999-88-777",
    "Count: 7",
    "Item #777 - Cost $3",
    "Order42 complete",
    "Tag55-Checked",
    "Out of stock - ID444",
    "Min pack of 12 - Code 5555",
    "Batch 003 - size: M",
    "Level 5 Clearance",
    "New - 888 units",
    "CodeX999"
])

# product_info.str.split(^'{1-9}', regex=True, expand=True)
# product_info.str.split('-', expand=True)

extracted_numbers = product_info.str.extract(r'(\d+)')
extracted_numbers

,0
0,1001
1,325
2,9988
3,15
4,3030
5,9007
6,2017
7,2
8,60
9,999


In [113]:
# Using a string column, create a series extracting the non-ASCII values.

non_ascii_data = pd.Series([
    "Café del Mar",
    "Niño jugando en la calle",
    "Über cool!",
    "München ist schön",
    "Résumé de l'été",
    "Привет, как дела?",
    "こんにちは世界",
    "مرحبا بالعالم",
    "你好，世界",
    "São Paulo - número 123",
    "Pokémon Go🔥",
    "Kosten: 25€",
    "Precio: 100¥",
    "© Copyright 2025",
    "🚀 Launch sequence initiated",
    "Frühstück um 8:00 Uhr",
    "Smiley 😊 approved",
    "Hèllo wørld",
    "Жизнь прекрасна",
    "مرحبا بكم في Python"
])

extracted_non_ascii = non_ascii_data.str.extract(r'([^\x00-\x7F]+)')
extracted_non_ascii

,0
0,é
1,ñ
2,Ü
3,ü
4,é
5,Привет
6,こんにちは世界
7,مرحبا
8,你好，世界
9,ã


In [144]:
# Using a string column, create a dataframe with the dummy columns for every character in the column.

# make.str.split(r'(.)', expand= True, regex=True)
make.str.findall(r'.').apply(pd.Series)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,A,l,f,a,,R,o,m,e,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,F,e,r,r,a,r,i,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D,o,d,g,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D,o,d,g,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,S,u,b,a,r,u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41139,S,u,b,a,r,u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41140,S,u,b,a,r,u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41141,S,u,b,a,r,u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41142,S,u,b,a,r,u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
